# SDXL Training / Inference Tutorial

### Note:
Currently, this notebook must be run in a NeMo container (> 24.09) and open_clip_torch<=2.24.0. An example command to launch the container:

```
docker run --gpus all -it --rm -v <your_nemo_dir>:/opt/NeMo -v <your_dataset_dir>:/datasets --shm-size=8g \
     -p 8888:8888 --ulimit memlock=-1 --ulimit \
      stack=67108864 <your_nemo_container>
```


## Introduction

This notebook illustrates how to train and perform inference using Stable Diffusion XL with the NeMo Toolkit. Despite differences in model configs, the training and inference procedure is similar as Stable Diffusion.

The implementation of Stable Diffusion XL is based on [SDXL: Improving Latent Diffusion Models for High-Resolution Image Synthesis](https://arxiv.org/abs/2307.01952).

This tutorial will guide you through the following topics:

1. Training a Stable Diffusion XL model.
2. Performing inference with the trained model.

## Datasets

Please refer to [Dataset Tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/multimodal/Multimodal%20Data%20Preparation.ipynb) for how to prepare a training dataset for Stable diffusion XL.

For a pre-cached Stable Diffusion dataset, each webdataset tar file should, at a minimum, include the pickle files that store the pre-cached image and text features:

```
t0_r0_0.tar
|---- 0000.pickle
|---- 0001.pickle
...
```

For non-precached Stable Diffusion dataset, each webdataset tar file should contain the raw texts and corresponding images:

```
t0_r0_0.tar
|---- 0000.jpg
|---- 0000.txt
|---- 0001.jpg
|---- 0001.txt
...
```

## Encoders Preparation

Depending on whether you precache the dataset, you might also need to first download the image and/or text encoders.

### Option 1: Training on Non-Precached Dataset (Use Encoders During Training)

#### A. Prepare VAE
To download the default VAE for Stable Diffusion:


In [ ]:
! wget https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/vae/diffusion_pytorch_model.safetensors
! mkdir -p /sdxl_ckpts
! mv diffusion_pytorch_model.safetensors /sdxl_ckpts/vae.safetensors

The above command will download the default VAE weights from HuggingFace and save it to `/sdxl_ckpts/vae.safetensors`.

**Note**: if you want to customize the saved location, make sure it is also reflected in your training config.
#### B. Prepare Text Encoder
For the text encoders used in Stable Diffusion XL, it will be automatically downloaded by the training script we provide.

The type of text encoder used in the sdxl model conditioner can be found in `conditioner_config` in the predefined training configs:

```
  conditioner_config:
    _target_: nemo.collections.multimodal.modules.stable_diffusion.encoders.modules.GeneralConditioner
    emb_models:
      - is_trainable: false
        input_key: captions
        ucg_rate: 0.1
        emb_model:
          _target_: nemo.collections.multimodal.modules.stable_diffusion.encoders.modules.FrozenCLIPEmbedder
          layer: hidden
          layer_idx: 11
      - is_trainable: false
        ucg_rate: 0.1
        input_key: captions
        emb_model:
          _target_: nemo.collections.multimodal.modules.stable_diffusion.encoders.modules.FrozenOpenCLIPEmbedder2
          arch: ViT-bigG-14
          version: laion2b_s39b_b160k
          freeze: true
          layer: penultimate
          always_return_pooled: true
          legacy: false
```


### Option 2: Training on Precached Dataset (Training UNet Only)

When using precached dataset (please refer to the [Dataset Tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/multimodal/Multimodal%20Data%20Preparation.ipynb) for details), every text feature and image feature are stored as key-value pairs in `.pickle` file:

```
{
 image_key: torch.Tensor(),
 text_key: torch.Tensor(),
}
```

Make sure in the training config, `cond_stage_key` is associated with `text_key` and `first_stage_key` is associated with `image_key`.

We offer an expample script to convert a dataset from `parquet` file to webdataset `tar` files at [parquet_conversion](https://github.com/NVIDIA/NeMo/blob/main/scripts/multimodal_dataset_conversion/parquet_conversion.py). Three different modes of prechaed training are provided, they are:

1. No Caching: VAE and Text encoders are loaded during training
2. Text only: Only text features are loaded from dataset during training
3. Both: Both image and text features are loaded from dataset during training

In each mode, the cached components should be saved in its raw format in tarfiles while cached components should be saved as torch.Tensor().

## Model Config Setup

Now we will begin setting up the config file needed for Stable Diffusion training. We will use [sd_train.yaml](https://github.com/NVIDIA/NeMo/blob/main/examples/multimodal/text_to_image/stable_diffusion/conf/sd_xl_base_train.yaml) as the template.

1. Modify `model.data.train.dataset_path` so that it has all the webdataset info files you want to train on
2. Modify `model.data.webdataset.local_root_path` to point to your dataset path
3. Make sure VAE path `model.first_stage_config.from_pretrained` is adjusted if using non-precached dataset
4. Make sure the `model.precache mode` is set properly with the dataset you prepared, as detailed above.
5. Configure `exp_manager.exp_dir` for experiment save directory
6. Configure `exp_manager.wandb_logger_kwargs` and/or `exp_manager.create_tensorboard_logger` if needed

**Note**: Please refer to NeMo Toolkit Developer Guide's Stable Diffusion page for more details on in-depth customizations, including all available optimizations.

## Training

Once everything is set up, training stable diffusion is as simple as running:


In [ ]:
! torchrun /opt/NeMo/examples/multimodal/text_to_image/stable_diffusion/sd_xl_train.py trainer.max_steps=100 model.data.train.dataset_path=/path/to/wdinfo.pkl model.data.webdataset.local_root_path=/path/to/dataset trainer.devices=1 trainer.num_nodes=1 model.micro_batch_size=1 model.global_batch_size=1 model.first_stage_config.from_pretrained=/sdxl_ckpts/vae.safetensors model.fsdp=False

Intermediate checkpoints (during training) and final checkpoint will be saved to `exp_manager.exp_dir` folder. Note that here we use synthetic data for demo purpose.

## Inference

Stable Diffusion XL inference needs a trained NeMo Stable Diffusion checkpoint, along with both the image encoder (VAE) and text encoder (CLIP). The checkpoint can be either a fully trained `.nemo` checkpoint or an intermediate checkpoint from training (typically in `.ckpt` format). 

### Inference Config Setup

Now we will begin setting up the config file needed for Stable Diffusion inference. We will use [sd_xl_infer_v2.yaml](https://github.com/NVIDIA/NeMo/blob/main/examples/multimodal/text_to_image/stable_diffusion/conf/sd_xl_infer_v2.yaml) as the template.

We generally use [Classifier Free Guidance](https://arxiv.org/abs/2207.12598) for better visual quality, which can be set at `sampling.base.scale`.

NeMo Stable Diffusion supports multiple samplers. Please refer to the developer guide for more details. Samplers can be set at `sampling.base.sampler`.

Inference supports a batch of text prompts, which can be set at `infer.prompt`. One can also generate a configurable number of images per prompt by setting `infer.num_samples`. Generated images will be saved to `out_path`.

You will also need to set the model checkpoint path at `model.restore_from_path` if you are loading from `.nemo` checkpoint, otherwise, mannually set `unet` checkpoints and `vae` checkpoint at `model.unet_config.from_pretrained` and `model.first_stage_config.from_pretrained`, respectively.

### Running the Inference

Once everything is set up, Stable Diffusion inference is as simple as running:

In [ ]:
! torchrun  /opt/NeMo/examples/multimodal/text_to_image/stable_diffusion/sd_xl_infer.py model.restore_from_path=/path/to/stable-diffusion-xl-train.nemo out_path=/sdxl_infer_out